# EDA for Prototype's Fire Data
#### Hannah Catri 217569841




In [20]:
import pandas as pd
from sodapy import Socrata
from datetime import datetime, timedelta

## Emergency Services Data Stream

#### Fire Department Calls for Service

In [2]:
# Get the number of rows in the dataset
# Save as a variable in case paging method of data retireval is needed
num=pd.read_json("https://data.sfgov.org/resource/RowID.json?$select=COUNT(incident_number)")
maxNumber=num.iloc[0,0]
print('The number of incidents in the set is',maxNumber)

The number of incidents in the set is 5642213


In [10]:
# Configure the Socrata client and create the generator that will retrieve the data
client = Socrata("data.sfgov.org", None)
#dataset_gen=client.get_all("nuek-vuh3")
dataset_gen=client.get("nuek-vuh3", limit=2000)

In [14]:
%%time
# Extract the dataset, this may take a while.
# Use shape to confirm the size matches the rows as per above maxNumber.
fire_cfs_df = pd.DataFrame.from_records(dataset_gen)
fire_cfs_df.shape
fire_cfs_df.sample(5)

CPU times: user 21.5 ms, sys: 3.03 ms, total: 24.6 ms
Wall time: 23.5 ms


,call_number,unit_id,incident_number,call_type,call_date,watch_date,received_dttm,entry_dttm,dispatch_dttm,response_dttm,...,unit_sequence_in_call_dispatch,fire_prevention_district,supervisor_district,neighborhoods_analysis_boundaries,rowid,case_location,:@computed_region_ajp5_b2md,on_scene_dttm,transport_dttm,hospital_dttm
1018,012160410,T06,01067014,Alarms,2001-08-04T00:00:00.000,2001-08-04T21:00:44.000,2001-08-04T21:00:20.000,2001-08-04T21:00:44.000,2001-08-04T21:04:39.000,2001-08-04T21:05:56.000,...,3,2,8,Mission,012160410-T06,"{'type': 'Point', 'coordinates': [-122.4220730...",20,NaN,NaN,NaN
1229,020140313,D2,02004312,Structure Fire,2002-01-14T00:00:00.000,2002-01-14T18:43:03.000,2002-01-14T18:42:47.000,2002-01-14T18:43:03.000,2002-01-14T18:43:12.000,2002-01-14T18:45:05.000,...,7,4,2,Marina,020140313-D2,"{'type': 'Point', 'coordinates': [-122.4409775...",13,NaN,NaN,NaN
796,013140180,RC1,01096456,Medical Incident,2001-11-10T00:00:00.000,2001-11-10T12:55:44.000,2001-11-10T12:54:26.000,2001-11-10T12:55:44.000,2001-11-10T12:55:57.000,NaN,...,3,3,6,South of Market,013140180-RC1,"{'type': 'Point', 'coordinates': [-122.4092454...",34,NaN,NaN,NaN
1926,022440389,T11,02072387,Structure Fire,2002-09-01T00:00:00.000,2002-09-01T19:36:56.000,2002-09-01T19:36:56.000,2002-09-01T19:36:56.000,2002-09-01T19:37:10.000,2002-09-01T19:38:16.000,...,2,6,9,Bernal Heights,022440389-T11,"{'type': 'Point', 'coordinates': [-122.4208936...",2,NaN,NaN,NaN
180,002960164,T48,00088719,Other,2000-10-22T00:00:00.000,2000-10-22T13:07:48.000,2000-10-22T13:07:01.000,2000-10-22T13:07:48.000,2000-10-22T13:08:01.000,2000-10-22T13:10:35.000,...,1,None,6,Treasure Island,002960164-T48,"{'type': 'Point', 'coordinates': [-122.3698008...",37,NaN,NaN,NaN


In [15]:
fire_cfs_df.describe()

,call_number,unit_id,incident_number,call_type,call_date,watch_date,received_dttm,entry_dttm,dispatch_dttm,response_dttm,...,unit_sequence_in_call_dispatch,fire_prevention_district,supervisor_district,neighborhoods_analysis_boundaries,rowid,case_location,:@computed_region_ajp5_b2md,on_scene_dttm,transport_dttm,hospital_dttm
count,2000,2000,2000,2000,2000,2000,2000,2000,2000,1144,...,2000,2000,2000,2000,2000,1998,1958,452,31,4
unique,1962,136,1962,16,934,1955,1962,1962,1962,1144,...,18,11,12,41,2000,1251,40,451,31,4
top,212230905,T03,21095490,Medical Incident,2001-05-04T00:00:00.000,2021-08-11T00:00:00.000,2021-08-11T09:33:10.000,2021-08-11T09:35:32.000,2021-08-11T09:36:25.000,2002-10-05T17:27:16.000,...,2,2,6,Tenderloin,022860320-T16,"{'type': 'Point', 'coordinates': [-122.4140860...",36,2001-12-21T00:49:40.000,2020-11-29T17:31:01.000,2020-11-29T17:57:52.000
freq,4,117,4,639,13,5,4,4,4,1,...,621,512,646,407,1,13,406,2,1,1


In [16]:
fire_cfs_df.call_type.unique()

array(['Alarms', 'Medical Incident', 'Structure Fire', 'Other',
       'Outside Fire', 'Water Rescue', 'HazMat', 'Traffic Collision',
       'Aircraft Emergency', 'Vehicle Fire', 'Electrical Hazard',
       'Fuel Spill', 'Industrial Accidents', 'Assist Police', 'Oil Spill',
       'Citizen Assist / Service Call'], dtype=object)

In [29]:
# set 72 hour lookback window
periodstart = datetime.now().isoformat()
periodend = (datetime.now()-timedelta(days=3)).isoformat()
query_str=("https://data.sfgov.org/resource/RowID.json?$where=call_date between '"+str(periodstart)+"' and '"+str(periodend)+"'")
print(query_str)
data=pd.read_json(query_str)
data.sample(5)

https://data.sfgov.org/resource/RowID.json?$where=call_date between '2021-09-01T21:02:10.712700' and '2021-08-29T21:02:10.712752'


InvalidURL: URL can't contain control characters. "/resource/RowID.json?$where=call_date between '2021-09-01T21:02:10.712700' and '2021-08-29T21:02:10.712752'" (found at least ' ')

2021-09-01T20:52:47.676131
2021-08-29T20:52:47.676184


In [ ]:
"https://data.sfgov.org/resource/RowID.json?$where=date between '2015-01-10T12:00:00' and '2015-01-10T14:00:00'